In [1]:
import pandas as pd
import nltk
import numpy as np
import re
import torch

In [2]:
train_df = pd.read_csv("train.csv")
print(train_df.shape)
print(train_df.head())

test_df = pd.read_csv("test.csv")
print(test_df.shape)
print(test_df.head())

(891, 12)
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN    

PREPROCESING

In [3]:
train_df = train_df.replace(np.nan, None)
test_df = test_df.replace(np.nan, None)

In [4]:
def normalize(df):

    def normalize_name(name):
        name = re.sub(r'[^a-zA-Z\s]','',name,re.I|re.A)
        name = nltk.word_tokenize(name)
        return name

    def normalize_ticket_number(ticket):
        return ticket.split(' ')[-1]

    def normalize_ticket_item(ticket):
        items = ticket.split(' ')
        if(len(items) == 1):
            return 'NONE'
        return ''.join(items[0:-1])

    df['Name'] = df['Name'].apply(lambda x : normalize_name(x))
    df['Ticket_number'] = df['Ticket'].apply(lambda x : normalize_ticket_number(x))
    df['Ticket_item'] = df['Ticket'].apply(lambda x : normalize_ticket_item(x)) 

In [5]:
normalize(train_df)
normalize(test_df)

In [6]:
train_labels = train_df['Survived']
train_features = train_df.drop(['Survived','Ticket'], axis = 1)

test_features = test_df.drop('Ticket', axis = 1)

In [7]:
train_features

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,3,"[Braund, Mr, Owen, Harris]",male,22.0,1,0,7.2500,None,S,21171,A/5
1,2,1,"[Cumings, Mrs, John, Bradley, Florence, Briggs...",female,38.0,1,0,71.2833,C85,C,17599,PC
2,3,3,"[Heikkinen, Miss, Laina]",female,26.0,0,0,7.9250,None,S,3101282,STON/O2.
3,4,1,"[Futrelle, Mrs, Jacques, Heath, Lily, May, Peel]",female,35.0,1,0,53.1000,C123,S,113803,NONE
4,5,3,"[Allen, Mr, William, Henry]",male,35.0,0,0,8.0500,None,S,373450,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"[Montvila, Rev, Juozas]",male,27.0,0,0,13.0000,None,S,211536,NONE
887,888,1,"[Graham, Miss, Margaret, Edith]",female,19.0,0,0,30.0000,B42,S,112053,NONE
888,889,3,"[Johnston, Miss, Catherine, Helen, ``, Carrie,...",female,None,1,2,23.4500,None,S,6607,W./C.
889,890,1,"[Behr, Mr, Karl, Howell]",male,26.0,0,0,30.0000,C148,C,111369,NONE


In [8]:
test_features

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,892,3,"[Kelly, Mr, James]",male,34.5,0,0,7.8292,None,Q,330911,NONE
1,893,3,"[Wilkes, Mrs, James, Ellen, Needs]",female,47.0,1,0,7.0,None,S,363272,NONE
2,894,2,"[Myles, Mr, Thomas, Francis]",male,62.0,0,0,9.6875,None,Q,240276,NONE
3,895,3,"[Wirz, Mr, Albert]",male,27.0,0,0,8.6625,None,S,315154,NONE
4,896,3,"[Hirvonen, Mrs, Alexander, Helga, E, Lindqvist]",female,22.0,1,1,12.2875,None,S,3101298,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"[Spector, Mr, Woolf]",male,None,0,0,8.05,None,S,3236,A.5.
414,1306,1,"[Oliva, y, Ocana, Dona, Fermina]",female,39.0,0,0,108.9,C105,C,17758,PC
415,1307,3,"[Saether, Mr, Simon, Sivertsen]",male,38.5,0,0,7.25,None,S,3101262,SOTON/O.Q.
416,1308,3,"[Ware, Mr, Frederick]",male,None,0,0,8.05,None,S,359309,NONE
